In [47]:
from commonfunctions import *

### Parameters

Annotation => filename ; y ; x ; x_min ; y_min ; x_max ; y_max ; category

In [48]:
IMAGE_SIZE = (256, 256)  # To resize all images to 64x64
DATASET_PATH = "TSRD_Train"  # Path to your dataset folder
ANNOTATION_FILE = "TSRD_Train_Annotation/TsignRecgTrain4170Annotation.txt"  # Path to the annotation file
SAVE_PATH = "processed_data.npz"  # File to save the processed dataset (NPZ to store NumPy arrays)
TEST_DATASET_PATH = "TSRD_Test"  # Path to your test dataset folder
TEST_ANNOTATION_FILE = "TSRD_Test_Annotation/TsignRecgTest1994Annotation.txt"  # Path to the test annotation file

### Helper functions

In [49]:
# Function to parse annotation file #####################################################
def parse_annotations(annotation_file):
    annotations = []
    with open(annotation_file, "r") as file:
        for line in file:
            parts = line.strip().split(";")
            filename = parts[0]
            y_axis = int(parts[1])
            x_axis = int(parts[2])
            bound_x_min = int(parts[3])
            bound_y_min = int(parts[4])
            bound_x_max = int(parts[5])
            bound_y_max = int(parts[6])
            label = int(parts[7])
            # if label == 54:
            #     continue
            annotations.append({
                "filename": filename,
                "y_axis": y_axis,
                "x_axis": x_axis,
                "bound_x_min": bound_x_min,
                "bound_y_min": bound_y_min,
                "bound_x_max": bound_x_max,
                "bound_y_max": bound_y_max,
                "label": label
            })
    return annotations


# Function to load images ################################################
def load_images(image_folder, annotations):
    images = []
    labels = []

    for ann in annotations:
        # Get the image path
        img_path = os.path.join(image_folder, ann["filename"])
        # Read the image
        img = io.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        images.append(img)
        labels.append(ann["label"])
    return images, labels

# get the acutal labels of the test dataset
def get_actual_labels(TEST_DATASET_PATH):
    actual_labels = []
    for filename in os.listdir(TEST_DATASET_PATH):
        label = int(filename.split("_")[0])
        actual_labels.append(label)
    return actual_labels

### Image Preprocessing

In [50]:
from skimage.morphology import disk, opening, closing
# Function to load and preprocess images ################################################
def preprocess_images(image_folder, annotations, image_size):

    # load images and labels
    images, labels = load_images(image_folder, annotations)

    processed_images = []

    for i in range(len(images)):
    # for i in range(10 , 20):

        ann = annotations[i]
        img = images[i]


        # # Crop the bounding box
        cropped_img = img[ann["bound_y_min"]:ann["bound_y_max"], ann["bound_x_min"]:ann["bound_x_max"]]

        # Convert to grayscale
        gray_image = cv.cvtColor(cropped_img, cv.COLOR_BGR2GRAY)

        # Apply Gaussian filter with sigma=3
        gaussian_blur = cv.GaussianBlur(gray_image, (5, 5), sigmaX=1, sigmaY=1)
        
        # Perform opening morphology using a disk-shaped structuring element
        structuring_element = disk(4)  # Disk of radius 5
        opened_image = opening(gaussian_blur, structuring_element)


        # # histogram equalization to all the channels
        # cropped_img[:, :, 2] = histogram_eq(cropped_img[:, :, 2])
        # cropped_img[:, :, 1] = histogram_eq(cropped_img[:, :, 1])
        # cropped_img[:, :, 0] = histogram_eq(cropped_img[:, :, 0])


        # # Convert to HSV
        # hsv = cv.cvtColor(cropped_img, cv.COLOR_RGB2HSV)

        # # Define Red Color Range
        # lower_red1 = np.array([0, 100, 100])
        # upper_red1 = np.array([10, 255, 255])
        # lower_red2 = np.array([160, 100, 100])
        # upper_red2 = np.array([180, 255, 255])

        # # Define Blue Color Range
        # lower_blue = np.array([70, 100, 100])
        # upper_blue = np.array([160, 255, 255])

        # # Create Red Masks
        # mask_red1 = cv.inRange(hsv, lower_red1, upper_red1)
        # mask_red2 = cv.inRange(hsv, lower_red2, upper_red2)
        # mask_red = mask_red1 + mask_red2  # Combine two red masks

        # # Create Blue Mask
        # mask_blue = cv.inRange(hsv, lower_blue, upper_blue)

        # red_pixels = np.sum(mask_red > 0)
        # blue_pixels = np.sum(mask_blue > 0)

        # if red_pixels > blue_pixels:
        #     print("Dominant Color: Red")
        # elif blue_pixels > red_pixels:
        #     print("Dominant Color: Blue")
        # else:
        #     print("No dominant color detected")

        # show_images([cropped_img, mask_red, mask_blue], ["Cropped Image", "Red Mask", "Blue Mask"])

        # # Convert to grayscale
        # gray_img = (rgb2gray(cropped_img)*255).astype(np.uint8)
        # print (gray_img)

        # # histogram equalization
        # hist_eq_img = histogram_eq(gray_img)
        # show_images([gray_img, hist_eq_img], ["Original", "Histogram Equalized"])

        # # increase the saturation of the image
        # hsv_img = cv.cvtColor(cropped_img, cv.COLOR_RGB2HSV)
        # hsv_img[:, :, 1] = hsv_img[:, :, 1] * 1.5
        # increased_saturation_img = cv.cvtColor(hsv_img, cv.COLOR_HSV2RGB)
        # show_images([cropped_img, increased_saturation_img], ["Original", "Increased Saturation"])

        # Resize the image
        resized_img = cv.resize(opened_image, image_size, interpolation=cv.INTER_CUBIC)

        # crop the circluar region with center equals the center of the bounding box and radius equals the half of the bounding box width
        mask1 = np.zeros_like(resized_img)
        center = (int(resized_img.shape[1]/2), int(resized_img.shape[0]/2))
        radius = int(resized_img.shape[1]/2)
        mask1 = cv.circle(mask1, center, radius, (255, 255, 255), -1)
        masked_img = cv.bitwise_and(resized_img, mask1)
        # show_images([img, resized_img ,  masked_img], ["Original", "Resized" , "Masked"])

        # Normalize pixel values
        # normalized_img = masked_img / 255.0
        normalized_img = masked_img

        # show_images([img, cropped_img, resized_img, normalized_img], ["Original", "Cropped", "Resized", "Normalized"])
        # show_images([img, cropped_img, gaussian_blur, opened_image, resized_img, normalized_img], ["Original", "cropped_img", "gaussian_blur", "opened_image", "resized_img",  "Normalized"])

        processed_images.append(normalized_img)

    # # Convert to NumPy arrays
    processed_images = np.array(processed_images)
    labels = np.array(labels)

    return processed_images, labels

### Feature Extraction

In [51]:
# Shape-Based Features (edge detection, Hough transform)

#canny edge detection
def canny_edge_detection(image):
    edges = cv.Canny((image * 255).astype(np.uint8), 40, 80)
    return edges

# Lines Hough transform
def lines_hough_transform(image):
    gray = rgb2gray(image)
    edges = canny_edge_detection(gray)
    lines = cv.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=45, minLineLength=20, maxLineGap=10)
    return lines

# Circles Hough transform
def circles_hough_transform(image):
    # gray = rgb2gray(image)
    edges = canny_edge_detection(image)
    circles = cv.HoughCircles(edges, cv.HOUGH_GRADIENT, dp=3, minDist=300, param1=50, param2=30, minRadius=30, maxRadius=150)
    return circles

# HOG
def extract_hog_features(image):
    # gray = rgb2gray(image)
    hog_features = hog(image, 
                       orientations=9, 
                       pixels_per_cell=(8, 8), 
                       cells_per_block=(2, 2), 
                       block_norm='L2-Hys', 
                       visualize=False, 
                       feature_vector=True)
    return hog_features


# Keypoint-Based Features

# Harris corner detection
def harris_corner_detection(image):
    gray = rgb2gray(image)
    gray = np.float32(gray)
    dst = cv.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)
    return dst

# SIFT 
def sift_features(image):
    # image = np.uint8(image * 255)
    # gray = rgb2gray(image)
    sift = cv.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors


### classificaion


In [52]:
# create vocabulary using KMeans clustering
def create_vocabulary(descriptors, k):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(descriptors)
    return kmeans

# generate BoVW histograms
def generate_bovw_histograms(descriptors_list, kmeans, k):
    histograms = []
    for descriptors in descriptors_list:
        if descriptors is not None:
            words = kmeans.predict(descriptors)
            histogram, _ = np.histogram(words, bins=np.arange(k+1), density=True)
        else:
            histogram = np.zeros(k)
        histograms.append(histogram)
    return np.array(histograms)

# generate BoVW histograms for one image
def generate_bovw_histograms_for_one_image(descriptors, kmeans, k):
    if descriptors is not None:
            words = kmeans.predict(descriptors)
            histogram, _ = np.histogram(words, bins=np.arange(k+1), density=True)
    else:
        histogram = np.zeros(k)
    return np.array(histogram)


### Train


In [53]:
def train ():
    
    # Parse annotations
    annotations = parse_annotations(ANNOTATION_FILE)

    # Load and preprocess images
    images, labels = preprocess_images(DATASET_PATH, annotations, IMAGE_SIZE)

    print("Succesfully preprocessed images")
    print("Extracting features...")

    # descreptors list for all images
    descriptors_list = []
    new_labels = []

    images_histograms = []

    for i in range(len(images)):
        img = images[i]
        label = labels[i]


        # Apply hog features
        hog = extract_hog_features(img)
        images_histograms.append(hog)

        # Apply Canny edge detection
        # edges = canny_edge_detection(img)

        # # Apply Hough transform for lines
        # lines = lines_hough_transform(img)
        # if lines is not None:
        #     for line in lines:
        #         x1, y1, x2, y2 = line[0]
        #         cv.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)


        # # Apply Hough transform for circles
        # detected_circles = circles_hough_transform(img)

        # if detected_circles is not None: 
        #     print ("detected_circles", detected_circles)

        #     # Convert the circle parameters a, b and r to integers. 
        #     detected_circles = np.uint16(np.around(detected_circles)) 

        #     for pt in detected_circles[0, :]: 
        #         a, b, r = pt[0], pt[1], pt[2] 

        #         # Draw the circumference of the circle. 
        #         cv.circle(img, (a, b), r, (0, 255, 0), 2) 

        #         # Draw a small circle (of radius 1) to show the center. 
        #         cv.circle(img, (a, b), 1, (0, 0, 255), 3) 

        # # apply Harris corner detection
        # dst = harris_corner_detection(img)
        # dst = cv.dilate(dst, None)
        # img[dst > 0.01 * dst.max()] = [0, 255, 0]

        # print ("img", img * 255)

        # # apply SIFT
        # keypoints, descriptors = sift_features(img)
        # if descriptors is not None:
        #     descriptors_list.append(descriptors)
        #     new_labels.append(label)


        # img=cv.drawKeypoints(img, keypoints,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # cv.imwrite('sift_keypoints.jpg',img)
        # show_images([img], ["SIFT"])

        # show_images([img, edges], ["Original", "Edges"])
        # print(f"Label: {label}")


    # # stack all descriptors
    # stack_descriptors = np.vstack(descriptors_list)

    # # create vocabulary using KMeans clustering
    # k = 290 # number of clusters (total number of keypoints / number of dataset images * number of classes) (Average)
    # kmeans = create_vocabulary(stack_descriptors, k)

    # print("descriptors_list_dimentions", len(descriptors_list))
    # print ("stack_descriptors_shape", stack_descriptors.shape)
    # print ("kmeans", kmeans)

    # pca = PCA(n_components=2)
    # reduced_data = pca.fit_transform(descriptors)

    # # Now separate the data, Note the flatten()
    # A = stack_descriptors[labels.ravel()==0]
    # B = stack_descriptors[labels.ravel()==1]

    # # Plot the data
    # plt.scatter(A[:,0],A[:,1])
    # plt.scatter(B[:,0],B[:,1],c = 'r')
    # plt.scatter(centers[:,0],centers[:,1],s = 80,c = 'y', marker = 's')
    # plt.xlabel('Height'),plt.ylabel('Weight')
    # plt.show()

    # Generate BoVW Histograms for Each Image
    # print("Generating BoVW histograms...")
    # images_histograms = generate_bovw_histograms(descriptors_list, kmeans, k)
    # print ("images_histograms_shape", images_histograms.shape)
    # for histogram in images_histograms:
    #     print ("histogram", histogram)
    #     print ("histogram_shape", histogram.shape)

    # images_histograms = []
    # for discriptor in descriptors_list:
    #     # Handle edge case where no descriptors are found
    #     if discriptor is None:
    #         discriptor = np.zeros((1, 128), dtype=np.float32)

    #     # Assign descriptors to the nearest cluster center (for each keypoint in the descriptor, find the nearest cluster center (nears word))
    #     words = kmeans.predict(discriptor)

    #     # Build histogram of visual words
    #     histogram = np.zeros(kmeans.n_clusters, dtype=np.float32)
    #     for word in words:
    #         histogram[word] += 1
    #     images_histograms.append(histogram)


    # standardize the features
    # images_histograms = StandardScaler().fit_transform(images_histograms)

    

    # Train SVM classifier
    svm = SVC(C=0.0008, kernel="linear", random_state=50)
    svm.fit(images_histograms, labels)


    return svm


### Predict


In [61]:
def predict (svm):

    predicted_labels = []

    actual_labels = get_actual_labels(TEST_DATASET_PATH)
    
    # try for the the blind test set folder
    # Load images
    blind_test_images = []
    for filename in os.listdir(TEST_DATASET_PATH):
        img_path = os.path.join(TEST_DATASET_PATH, filename)
        img = io.imread(img_path)
        blind_test_images.append(img)



    # # Preprocess images
    # blind_test_images = np.array(blind_test_images)
    # blind_test_images = blind_test_images / 255.0

    # Extract SIFT features
    blind_test_descriptors_list = []
    # print("imgs shape", len(blind_test_images))
    # print("actual labels", len(actual_labels))

    # for i, img in enumerate(blind_test_images):
    # for i in range(400, 450):
    for i in range(len(blind_test_images)):
        img = blind_test_images[i]

        # if actual_labels[i] == 54:
        #     continue

        # image resizing
        img = cv.resize(img, IMAGE_SIZE, interpolation=cv.INTER_CUBIC)

        gray_image = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Apply Gaussian filter with sigma=3
        gaussian_blur = cv.GaussianBlur(gray_image, (7, 7), sigmaX=1, sigmaY=1)
        
        # Perform opening morphology using a disk-shaped structuring element
        structuring_element = disk(4)  # Disk of radius 5
        opened_image = opening(gaussian_blur, structuring_element)

        # hough circle
        detected_circles = circles_hough_transform(opened_image)

        # crop the image to the circular region if exists
        if detected_circles is not None:
            # print ("detected_circles", detected_circles)
            for pt in detected_circles[0, :]: 
                a, b, r = pt[0], pt[1], pt[2] 
                # print ("a, b, r", a, b, r)
                mask1 = np.zeros_like(img)
                mask1 = cv.circle(mask1, (int(a), int(b)), int(r), (255, 255, 255), -1)
                masked_img = cv.bitwise_and(img, mask1)
                img = masked_img

        # if detected_circles is not None: 
        #     # print ("detected_circles", detected_circles)

        #     # Convert the circle parameters a, b and r to integers. 
        #     detected_circles = np.uint16(np.around(detected_circles)) 

        #     for pt in detected_circles[0, :]: 
        #         a, b, r = pt[0], pt[1], pt[2] 

        #         # Draw the circumference of the circle. 
        #         cv.circle(img, (a, b), r, (0, 255, 0), 2) 

        #         # Draw a small circle (of radius 1) to show the center. 
        #         cv.circle(img, (a, b), 1, (0, 0, 255), 3)

        # show_images([img], ["Hough Circles"]) 


        # # apply SIFT
        # keypoints, descriptors = sift_features(img)
        # # blind_test_descriptors_list.append(descriptors)

        # # img=cv.drawKeypoints(img, keypoints,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # # cv.imwrite('sift_keypoints.jpg',img)
        # # show_images([img], ["SIFT"])

        # # Handle edge case where no descriptors are found
        # if descriptors is None:
        #     descriptors = np.zeros((1, 128), dtype=np.float32)

        # # Assign descriptors to the nearest cluster center (for each keypoint in the descriptor, find the nearest cluster center (nears word))
        # words = kmeans.predict(descriptors)

        # # Build histogram of visual words
        # histogram = np.zeros(kmeans.n_clusters, dtype=np.float32)
        # for word in words:
        #     histogram[word] += 1

        # Normalize histogram
        # histogram /= len(words)
        # print ("histogram", histogram)
        # img = img / 255.0
        hog = extract_hog_features(opened_image)

        # predict the label
        predicted_label = svm.predict(hog.reshape(1, -1))
        predicted_labels.append(predicted_label[0])
        # print(f"Predicted label: {predicted_label[0]}")

    return predicted_labels

# Test Predict with one image

In [62]:
def predict_image(svm, img):
    """
    Predict the label of a single image using the given SVM model.

    Args:
        svm: Trained SVM model.
        img: Input image as a numpy array.

    Returns:
        Predicted label for the input image.
    """
    # Resize the image to match the expected dimensions
    img = cv.resize(img, IMAGE_SIZE, interpolation=cv.INTER_CUBIC)

    # Convert to grayscale
    gray_image = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Apply Gaussian filter with sigma=3
    gaussian_blur = cv.GaussianBlur(gray_image, (7, 7), sigmaX=1, sigmaY=1)

    # Perform opening morphology using a disk-shaped structuring element
    structuring_element = disk(4)  # Disk of radius 4
    opened_image = opening(gaussian_blur, structuring_element)

    # Perform Hough Circle Transform
    detected_circles = circles_hough_transform(opened_image)

    # Crop the image to the circular region if circles are detected
    if detected_circles is not None:
        for pt in detected_circles[0, :]: 
            a, b, r = pt[0], pt[1], pt[2]
            mask = np.zeros_like(img)
            mask = cv.circle(mask, (int(a), int(b)), int(r), (255, 255, 255), -1)
            img = cv.bitwise_and(img, mask)

    # Extract HOG features
    hog = extract_hog_features(opened_image)

    # Predict the label
    predicted_label = svm.predict(hog.reshape(1, -1))

    return predicted_label[0]


In [56]:
# from skimage import io

# # Load the image (replace 'path_to_image.jpg' with the actual path)
# image = io.imread("TSRD_Test_1/022_1_0005_1_j.png")

# # Predict the label
# label = predict(svm, image)
# print(f"Predicted label: {label}")

### Run the Training phase


In [57]:
svm = train()

Succesfully preprocessed images
Extracting features...


#### Main (predict test images)


In [63]:
from sklearn.metrics import classification_report, accuracy_score

# Main execution
if __name__ == "__main__":
    # Predict the labels of the test dataset
    predicted_labels = predict(svm)
    print ("predicted_labels", predicted_labels)

    # Get the actual labels
    actual_labels = get_actual_labels(TEST_DATASET_PATH)

    # Filter out labels equal to 54 and corresponding predictions
    filtered_actual_labels = actual_labels

    # for i in range (len(actual_labels)):
    #     if actual_labels[i] != 54:
    #         filtered_actual_labels.append(actual_labels[i])

    # Evaluate Classifier
    print("Filtered actual_labels   ", filtered_actual_labels)
    print("Filtered predicted_labels", predicted_labels)

    if len(filtered_actual_labels) == len(predicted_labels):
        accuracy = accuracy_score(filtered_actual_labels, predicted_labels)
        print(f"Test Accuracy: {accuracy * 100:.2f}%")
        print(classification_report(filtered_actual_labels, predicted_labels))
    else:
        print("Error: Length mismatch between filtered labels!")


predicted_labels [22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 26, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 24, 24, 26, 26, 26, 24, 26, 22, 22, 22, 22, 22, 22, 22, 22, 26, 22, 22, 22, 22, 22, 22, 22, 26, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 26, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 26, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25

In [ ]:
import joblib

# Save the trained model
joblib.dump(svm, 'trained_model.pkl')
print("Model saved as 'trained_model.pkl'")


Model saved as 'trained_model.pkl'


# Load model and predict

In [172]:
# Load the trained model
svm = joblib.load('trained_model.pkl')

# Load the image
image_path = "TSRD_Test_1/022_1_0004_1_j.png"
img = cv.imread(image_path)

if img is None:
    print(f"Failed to load image at {image_path}")
else:
    # Predict the label
    label = predict_image(svm, img)
    print(f"Predicted label: {label}")

Predicted label: 22


In [27]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2 as cv
import joblib
import numpy as np
from skimage.morphology import disk, opening
from skimage.feature import hog


IMAGE_SIZE = (256, 256)

# Circles Hough transform
def circles_hough_transform(image):
    # gray = rgb2gray(image)
    edges = canny_edge_detection(image)
    circles = cv.HoughCircles(edges, cv.HOUGH_GRADIENT, dp=3, minDist=300, param1=50, param2=30, minRadius=30, maxRadius=150)
    return circles

#canny edge detection
def canny_edge_detection(image):
    edges = cv.Canny((image * 255).astype(np.uint8), 40, 80)
    return edges

# HOG
def extract_hog_features(image):
    # gray = rgb2gray(image)
    hog_features = hog(image, 
                       orientations=9, 
                       pixels_per_cell=(8, 8), 
                       cells_per_block=(2, 2), 
                       block_norm='L2-Hys', 
                       visualize=False, 
                       feature_vector=True)
    return hog_features

def predict_image(svm, img):
    """
    Predict the label of a single image using the given SVM model.

    Args:
        svm: Trained SVM model.
        img: Input image as a numpy array.

    Returns:
        Predicted label for the input image.
    """
    # Resize the image to match the expected dimensions
    img = cv.resize(img, IMAGE_SIZE, interpolation=cv.INTER_CUBIC)

    # Convert to grayscale
    gray_image = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Apply Gaussian filter with sigma=3
    gaussian_blur = cv.GaussianBlur(gray_image, (7, 7), sigmaX=1, sigmaY=1)

    # Perform opening morphology using a disk-shaped structuring element
    structuring_element = disk(4)  # Disk of radius 4
    opened_image = opening(gaussian_blur, structuring_element)

    # Perform Hough Circle Transform
    detected_circles = circles_hough_transform(opened_image)

    # Crop the image to the circular region if circles are detected
    if detected_circles is not None:
        for pt in detected_circles[0, :]: 
            a, b, r = pt[0], pt[1], pt[2]
            mask = np.zeros_like(img)
            mask = cv.circle(mask, (int(a), int(b)), int(r), (255, 255, 255), -1)
            img = cv.bitwise_and(img, mask)

    # Extract HOG features
    hog = extract_hog_features(opened_image)

    # Predict the label
    predicted_label = svm.predict(hog.reshape(1, -1))

    
        # Map label to prediction
    label_map = {
        22: "Turn left",
        24: "Turn right",
        25: "Keep left",
        26: "Keep right",
        55: "Stop white bar",
        56: "Stop sign"
    }

    return label_map.get(predicted_label[0], "Unknown")

    # return predicted_label[0]


def load_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if not file_path:
        return

    try:
        img = cv.imread(file_path)
        if img is None:
            raise ValueError("Failed to load the image.")

        # Convert to RGB for displaying
        img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(img_pil)

        # Update GUI with the image
        image_label.configure(image=img_tk)
        image_label.image = img_tk
        image_label.file_path = file_path

        file_name_label.config(text=f"Loaded: {file_path.split('/')[-1]}")

        predict_button.config(state=tk.NORMAL)
    except Exception as e:
        messagebox.showerror("Error", f"Could not load image: {e}")

def make_prediction():
    try:
        file_path = image_label.file_path
        img = cv.imread(file_path)

        # Predict the label
        label = predict_image(svm, img)

        # map images

        # Display the result
        result_label.config(text=f"Prediction: {label}")
    except Exception as e:
        messagebox.showerror("Error", f"Prediction failed: {e}")

# Initialize GUI
root = tk.Tk()
root.title("Image Prediction")
root.geometry("500x500")

# Load SVM model
svm = joblib.load('trained_model.pkl')

# GUI components
frame = tk.Frame(root)
frame.pack(pady=10)

file_name_label = tk.Label(frame, text="No image loaded", font=("Arial", 14))
file_name_label.pack()

image_label = tk.Label(frame)
image_label.pack()

load_button = tk.Button(root, text="Load Image", command=load_image, font=("Arial", 12))
load_button.pack(pady=5)

predict_button = tk.Button(root, text="Predict", command=make_prediction, font=("Arial", 12), state=tk.DISABLED)
predict_button.pack(pady=5)

result_label = tk.Label(root, text="Prediction: None", font=("Arial", 14))
result_label.pack(pady=10)

# Run the GUI
root.mainloop()
